In [1]:
import jieba
import pandas as pd
df_pos = pd.read_csv('./data/pos.csv', encoding = 'utf-8', sep='\t')
df_pos = df_pos.dropna()

df_neg = pd.read_csv('./data/neg.csv', encoding = 'utf-8', sep='\t')
df_neg = df_neg.dropna()


pos = df_pos.content.values.tolist()[1000:51000]
neg = df_neg.content.values.tolist()[1000:51000]

In [2]:
stopwords = pd.read_csv('./rowdata/stopwords.txt', 
                        index_col=False, quoting=3, sep='\t', names=['stopword'], encoding='utf-8')
stopwords = stopwords['stopword'].values

In [3]:
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append((" ".join(segs), category))
        except Exception:
            continue
sentences = []

preprocess_text(pos, sentences, 'pos')
preprocess_text(neg, sentences, 'neg')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/tb/k5xqd901235gk634ymvtk5140000gn/T/jieba.cache
Loading model cost 0.867 seconds.
Prefix dict has been built successfully.


In [4]:
import random
random.shuffle(sentences)

In [5]:
from sklearn.model_selection import train_test_split
x, y = zip(* sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1234)

In [6]:
len(x_train)

75000

In [8]:
len(x_test)

25000

In [9]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.91432

In [10]:
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


class TextClassifier():

    def __init__(self, classifier=SVC(kernel='linear')):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=12000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [11]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['pos']
0.92092


In [12]:
def comPreProcess(text):
    sentence = []
    segs = jieba.lcut(text)
    segs = filter(lambda x: len(x)>1, segs)
    segs = filter(lambda x: x not in stopwords, segs)
    sentence.append(' '.join(segs))
    return sentence[0]

comPreProcess("你是个大笨蛋，我讨厌你，你可太过分了啊")

'笨蛋 讨厌 太过分'

In [13]:
df_pl_content = pd.read_csv('./data/BiliComment_contain20000.csv', sep='\t')

df_pl_content = df_pl_content.content.values.tolist()

for i in df_pl_content:
    print(i,text_classifier.predict(comPreProcess(i)))  # 除了分词还可以过滤下 停用词，写个函数啥的

也是上过央视哈哈哈哈哈哈 ['pos']
恶意抢注 ['pos']
加油 ['pos']
老爹说过要用魔法对抗魔法        出自《成龙历险记》。 ['pos']
支持维权 ['pos']
身份证不见了还可以去改名吗？急，打算明天去公安局改的，结果身份证不见了，真的想哭，已经成年了，只带户口本可以去改吗？ ['neg']
加油 ['pos']
真他妈的可悲 ['pos']
2507.4w ['pos']
一杯敬死神，一杯敬汉卿。 ['pos']
加油 ['pos']
加油 ['pos']
皮皮虾和b站都关注了 ['pos']
加油⛽️ ['pos']
冲冲冲 ['pos']
这事本身就很荒唐，人家名字也可以注册？呵呵笑死我了，这个部门在干嘛 ['neg']
真的没想到敬汉卿是真名，父母太有才了，取的好好听 ['pos']
第一次三连[OK] ['pos']
太难了up主 ['pos']
一直喜欢敬汉卿大大的视频，支持依法维权，坚决抵制这种不道德的行为。 ['pos']
支持你 ['pos']
敬汉卿你好，我也经常看你的节目，我们也是做商标这块的。刚才查询了一下，目前这个41还不是最要命的，你的35类也被抢注了，现在正在公示，如果35类被抢注，你就不能做广告，彻底凉凉了，希望你能引起共重视，现在赶紧去。35类的商标进行复议 复制自热心网友35类已经在2019.7.13号开始初审公示了，如果在三个月之内，也就是10月13号之前没有人复议的话就通过了，你35类商标如果丢了的话，你彻底就告别这个名字了（复制粘贴评论区）敬汉卿你好，我也经常看你的节目，我们也是做商标这块的。刚才查询了一下，目前这个41还不是最要命的，你的35类也被抢注了，现在正在公示，如果35类被抢注，你就不能做广告，彻底凉凉了，希望你能引起共重视，现在赶紧去。35类的商标进行复议 复制自热心网友35类已经在2019.7.13号开始初审公示了，如果在三个月之内，也就是10月13号之前没有人复议的话就通过了，你35类商标如果丢了的话，你彻底就告别这个名字了（复制粘贴评论区） ['neg']
抵制非法经营，保护维护自名正当权益，保护姓名并提供法律法规政策支持。敬汉卿，加油！[打call] ['neg']
奥利给 ['pos']
加油[微笑] ['pos']
就服你_x1f44d__x1f44d__x1f44d

麻烦大家顶上去，让更多UP主可以看到。@欣小萌有点污 ['neg']以有效杜绝各种搬运侵权问题。 品所有权的唯一有效法律途径。（唯一可能言重了，但绝对是强力手段）
坚决维权_x1f44d__x1f44d__x1f44d_ ['neg']
加油 ['pos']
支持合法维权 ['pos']
加油奥力给 ['pos']
加油 ['pos']
支持。 ['pos']
加油(ง •̀_•́)ง ['pos']
支持 ['pos']
加油加油 ['pos']
支持支持！ ['pos']
红红火火恍恍惚惚的几秒央视 ['pos']
支持 ['pos']
飞起！ ['pos']
哈哈哈哈哈哈哈 ['pos']
邓紫棋也是一样 ['pos']
来来来，支持 ['pos']
加油 ['pos']
支持[热词系列_奥力给] ['pos']
支持合法维权！！ ['pos']
支持合法维权！！！ ['pos']
支持合法维权！！！！ ['pos']
加油！ ['pos']
加油(ง •̀_•́)ง ['pos']
加油啊 ['pos']
出名upupup ['pos']
永远支持正义！ ['pos']
等等这个老干妈冰棍什么时候的事？！ ['pos']
考古 ['pos']
冲 ['pos']
无意间又刷到了这条视频，去看看我有没有被注册（工具：《天眼查》） ['neg']
怎么又挖出来了 ['pos']
2457w ['pos']
加油 ['pos']
加油↖(^ω^)↗ ['pos']
硬币  恐怖啊 ['neg']
看民法和刑法，《民法》侵权部分，《刑法》看看侵权后是否涉及刑事非法侵占，和侵占后是否做过犯罪案件。 ['neg']
加油！！！！！ ['pos']
dd ['pos']
加油加油加油！！！！！！[打call][打call][打call][打call] ['pos']
加油啊up，支持维权 ['pos']
国内恶意抢注商标很恶心，应该立法严惩 ['neg']
加油 ['pos']
我的两家公司在北京注册的名字内容都被人偷了，对于我来说损失很大（我都心凉了，人命案都维权不了，更何况公司的知识产权呢），我家的刑事犯罪侵害案件法律维权都得不到，希望你维权成功，但做好心理准备，中国的法律很无耻恶心！ ['neg']
up加油！ ['pos']
起名叫王者荣耀的孩子怎么办 ['neg

点个赞，评个论，收个藏[热词系列_可以][热词系列_三连] ['pos']
加油 ['pos']
打1 ['pos']
上去！！ ['pos']
加油！！！ ['pos']
nb ['neg']
加油 ['pos']
支持支持 ['pos']
总有一些垃圾为了钱毫无底线 ['neg']
加油 ['pos']
加油啊啊啊！看好你！ ['neg']
全b站最高投币 ['pos']
支持up ['pos']
up大大加油 ['pos']
加油，我支持。 ['pos']
nbnb，逼敬汉卿改名，这厂商怎么不上天 ['neg']
公司地址安徽省芜湖市镜湖区万达广场二期3#楼1724室《没想引战》 ['pos']
牛 ['pos']
奥利给！肝 ['pos']
敬汉卿，老子顶你！[支持] ['neg']
好久以前就在微博看到的敬汉卿的视频，当时觉得这大哥傻愣愣的，就特别映像深刻[囧] ['pos']
支持合法维权 ['pos']
支持支持支持 ['pos']
查商标注册的网站是什么来着？ ['pos']
加油(ง •̀_•́)ง ['pos']
评论 ['pos']
评论评论 ['pos']
加油加油！！ ['pos']
加油 ['pos']
那根本就不是正儿八经的公司，就是看你火敲诈你的 ['pos']
加油！支持支持支持 ['pos']
来了 ['pos']
[微笑] ['pos']
没事，没事。！我挺你！这是你的本名，有人去抢注那肯定是不行的！算是抢你的名字了。这样现在肯定很没法。 ['neg']
你非常出名 ['pos']
冲冲冲 ['pos']
评论 ['pos']
顶 ['pos']
支持支持 ['pos']
合法维权 ['neg']
冲冲冲 ['pos']
支持维权。 ['pos']
竟然才97年? ['pos']
支持合法维权！ ['pos']
支持合法维权[打call] ['pos']
黎明卿 ['pos']
卧槽，这视频播放几千万了，都超过大部分十二集番剧了，敬汉卿这波赚的盆满钵满[doge] ['neg']
冲 ['pos']
我是碰巧是做商标服务商，这种情况很简单的，查一下她的商标注册时间，你这个视频里提供的证据，足够可以撤销她的商标。要是你自己维权，可以去告她赔偿，不过中国这个力度不够，觉得撤销她商标就好了 ['neg']
可不可以以勒索罪反诉？ ['

敬汉卿加油，我支持你。 ['pos']
时隔四个月，国家有关部门也出台相关政策了。国家市场监督管理总局审议通过《规范商标申请注册行为若干规定》，自2019年12月1日起施行。《规定》明确，申请商标注册不得有下列行为：不以使用为目的恶意申请商标注册的；复制、摹仿或者翻译他人驰名商标的；基于合同、业务往来关系或者其他关系明知他人在先使用的商标存在而申请注册该商标的；损害他人现有的在先权利或者以不正当手段抢先注册他人已经使用并有一定影响的商标的等。此外，注册商标没有正当理由连续三年不使用的，任何单位或者个人可以向商标注册部门申请撤销该注册商标。 ['neg']
推荐点进来的，没看时间，傻乎乎的以为是最新的 ['neg']
2015年从百度上关注的敬汉卿，本想着也就是随便玩玩背景什么的也没有好到哪去，应该不会被抄袭吧，之后玩了B站，后来发现敬汉卿也在这上面，也没想到会有这么多粉丝，但唯一能猜到的是的却会被抄袭，但没想到被抄袭得人还那么狂，毕竟人红事非多吗(#／。＼#) ['pos']
[打call][吃瓜] ['pos']
镇站之宝 ['pos']
11 ['pos']
加油加油 ['pos']
加油 ['pos']
倒过来告那家公司侵权，使用了自己的名字[doge] ['pos']
要用魔法打败魔法[滑稽] ['pos']
支持 ['pos']
噶油 ['pos']
支持 ['pos']
请告到它破产 ['pos']
加油 ['pos']
加油_x1f4aa_ ['pos']
按照二十万播放量八百块来计算，两千多万，老汉能拿多少钱？ ['pos']
加油熬 ['pos']
我要赶紧评论几个 ['neg']
B站推送就点进来看了一下，谁能给我科普一下这是出了什么事？ ['neg']
维护利益有风险，报道需谨慎； ['neg']假第一人蒋卫锁遇害；和果冻&#39;而被淡出将近半年；
这种公司我不会买他们家的产品 ['pos']
支持 ['pos']
不客气 ['pos']
支持 ['pos']
维权 ['neg']
支持 ['pos']
现在咋样了 ['neg']
[点赞][点赞][点赞] ['pos']
加油冲冲冲 ['pos']
_(:з」∠)_ ['pos']
点赞投币，祝up主成功！ ['pos']
汉卿帅 ['pos']
这个公司太无耻了。抢注火热的up主名字。 [

不晓得说什么，但希望阿婆主挺住加油！恶势力真的太猖狂了！支持维权！同时也感谢你出这个视频提醒大家！ ['pos']
支持[微笑] ['pos']
支持 ['pos']
来了！！！！ ['pos']
弹幕里刷数字合影的，如果你们不需要大脑的话可以捐给有需要的人 ['neg']
加油，支持up主 ['pos']
你很出名了，小卿卿 ['pos']
超火卿卿 ['pos']
支持维权 ['pos']
你很有名 ['pos']
点关注 ['pos']
都说走什么事要走法律程序，但是有时候法律是真的扯卵谈 ['neg']
敬汉卿很出名！！！ ['pos']
加油，支持维权！！ ['pos']
可以哦，加油 ['pos']
哈哈 ['pos']
凭啥要改啊，真是小人太多了[囧][囧] ['pos']
emmmmmm悄咪咪留个言吧 ['pos']
评论 ['pos']
支持维权，给你增加知名度！ ['pos']
加油加油 ['pos']
鼓励 ['pos']
什么人都有…… ['pos']
加油！！！！！！ ['pos']
已经过千万了 ['pos']
后续怎样了 ['pos']
[嫌弃] ['pos']
支持up主 ['pos']
卿汉敬也被抢注了，没希望了[偷笑] ['pos']
go ['pos']
[2233娘_吃惊][2233娘_吃惊][2233娘_吃惊][2233娘_吃惊][2233娘_吃惊][2233娘_吃惊] ['pos']
加油加油 ['pos']
支持[支持] ['pos']
这种恶意注册的公司我觉得就应该把他们个人征信挂钩，看他们还敢不敢作妖[吐]恶臭这种公司 ['pos']
加油 ['pos']
顶顶顶 ['pos']
顶！ ['pos']
名场面好好笑 ['pos']
公道自在人心！ ['pos']
加油 ['pos']
对不起，按照国家法律，名字的使用权大于商标使用权 ['neg']
一个投币比点赞多的视频 ['neg']
加油 ['pos']
原来这么好听的名字就是你的真名。话说就算是名字也是要注册商标才有权益保护的。 ['neg']
[酸了][酸了] ['pos']
加油 ['pos']
支持老哥合法维权 ['pos']
敬汉卿！！！加油！！！我们要坚决地同违法犯罪作斗争！！！ ['pos']
18.5万评论，emmmmmmmmm[doge] ['

...

认识的上一个千万是白鼠 ['pos']
举报字体侵权 ['pos']
即将成为镇站之宝了！！！ ['pos']
加油！一定会胜利！ ['pos']
支持 ['pos']
支持啊敬，支持搞笑卿 ['pos']
破站不倒，陪你到老。 ['pos']
支持 ['pos']
以后up们得弄些奇葩而且复杂奇怪的名字了哦 ['neg']
冲鸭 ['pos']
一战封神 ['pos']
关注up主这么久了才知道我跟死神之子是同年同月同日_x1f605__x1f605__x1f605_ ['pos']
声援UP，支持维权 ['pos']
加油！AV62521889 ['pos']
支持维权！ ['pos']
原来你真名就叫这个，哇，很特别唉，羡慕 ['neg']
第一次看到点击率这么高的 ['neg']
UID:94513136 这个人 无良公司官方号 ['neg']
支持合法维权 ['pos']
加油 ['pos']
支持你 ['pos']
加油吧。 ['pos']
支持合法维权 ['pos']
对方已经拿到证书了，所以首先提出无效宣告（理由:恶意抢注），准备相应材料证明材料。 ['neg']
加油啊 ['pos']
加油 ['pos']
UP主加油！ ['pos']
突然想炸掉那个公司[doge] ['pos']
加油 ['pos']
死神安排，以法胜法 ['neg']
敬汉卿居然比我还小（不这不是重点 ['pos']
加油＾０＾~ ['pos']
加油啊兄弟 ['pos']
460W粉涨了70W，播放量1400W加油＾０＾~ ['pos']
哈哈哈哈哈哈加油！ ['pos']
挺你兄弟[支持] ['pos']
咱们B站整个排面全都给你撑腰[点赞]拿起法律的武器 ['pos']
几秒央视敬汉卿[滑稽] ['pos']
我擦！！！97年的 好成熟的大葛格哦[画风突变]人家96年的都没他成熟 ['neg']
！% ['pos']
up，加油（￣▽￣）！ ['pos']
支持支持支持，用起法律武器！ ['pos']
加油！ ['pos']
好，我来证明了 ['pos']
up加油 ['pos']
加油up主。 ['pos']
支持 ['pos']
打个气 ['neg']
敬汉卿之所以会被鬼畜是因为他人气高吧 ['neg']
加油 ['pos']
等着，我找到那个公司了，明天组团炸了它

支持维权！ ['pos']
加油！！！ ['pos']
三百多万的投币，太恐怖了 ['neg']
有个问题，如果对方抢注了，又过了三个月，是不是就很难拿回来了？ ['neg']
给你打气 ['pos']
支持 ['pos']
出名？！！ ['pos']
共青团中央今天看到了_x1f601_ ['pos']
加油！！！ ['pos']
支持！ ['pos']
来了 ['pos']
公道自在人心，支持合法维权。[支持] ['pos']
加油加油(ง •̀_•́)ง ['pos']
你真的好出名 ['neg']
打气 ['pos']
加油 ['pos']
加油！ ['pos']
加油！ ['pos']
支持维权 ['pos']
加油加油，无论怎样都会继续关注的[点赞] ['pos']
大家好，我是敬汉敬[大笑] ['pos']
支持你 ['pos']
加油_x1f4aa_ ['pos']
这些坏人 ['pos']
加油，别输啊 ['pos']
我透！敬汉卿只比我大三岁！ ['pos']
潜水党顶一下 ['pos']
天眼查也看到几个敬汉卿商标信息 ['neg']
加油敬汉卿！我们永远支持你！ ['pos']
敬汉卿加油 ['pos']
一开始是HQ先生吧 ['pos']
这个投币和播放量全站最高了吧（￣▽￣） ['pos']
上央视了就别想不出名了[奸笑] ['pos']
没想到吧，劳资用的是真名[doge] ['neg']
加油 ['pos']
up，up。看这个视频，可以反告那些公司使用盗版字体，以其人之道还治其身。把我顶上去，让up看到。https://b23.tv/av62311904[doge] ['neg']
加油呀！ ['pos']
加油！ ['pos']
告你的都是神经病 ['pos']
来了来了 ['pos']
你很知名了 ['pos']
支持支持 ['pos']
来了来了来了 ['pos']
恭喜这家公司，被共青团盯上了 ['pos']
[微笑][微笑]支持 ['pos']
每天给jhq投两颗硬币，一定要要胜诉，支持你(°∀°)ﾉ ['pos']
我能评论啦 ['neg']
加油啊 ['pos']
加油 ['pos']
1360w ['pos']
加油加油加油 ['pos']
还在榜一！ ['pos']
加油 ！ ['pos']
帮出名+1[doge]

讹诈这样努力的人，良心不会痛吗？！ ['pos']
卧槽过千万了 ['neg']
我想知道最后结果如何 ['pos']
加油 ['pos']
敬汉卿，加油啊！ ['pos']
up加油！我们支持你！[支持] ['pos']
up加油！ ['pos']
加油加油！！！ ['pos']
敬汉卿加油！！！！ ['pos']
加油，支持依法维权 ['pos']
加油加油！！ ['pos']
加油！ ['pos']
多找几家媒体曝光，起个标题“惊！起名字还要给钱” ['neg']
加油 ['pos']
合法维权[支持] ['pos']
支持 ['pos']
支持维权 ['pos']
2333 ['pos']
支持 ['pos']
加油＾０＾~ ['pos']
刚把嘚(加油)[doge]！ ['pos']
加油！ ['pos']
咋不更新视频了？ ['pos']
159002(=・ω・=) ['pos']
公民享有姓名权，有权决定、使用和依照规定改变自己的姓名，禁止他人干涉、盗用、假冒。 ['neg']经使用并有一定影响的商标。
这两天这个事情的热度已经爆了，支持你！！ ['neg']
我17年在虎牙上看到过你的视频[喜欢]b站也关注很久了 ['pos']
看了那家公司的消息，他居然说吧平台老板玩进去可就亏大了，居然想把睿总玩进去呵呵[抠鼻][抠鼻][抠鼻][抠鼻] ['pos']
加油 ['pos']
jhq牛逼 ['pos']
平均一手[微笑][微笑][微笑] ['pos']
流氓都开始懂法律了，难搞哦。 ['neg']
必胜 ['pos']
哇！这也太过分了！ ['neg']
目测很多被抢注，都是河北冰川，估计是同一家会计公司。 ['neg']
敬汉卿加油 ['pos']
支持 ['pos']
加油支持你 ['pos']
看了下大体上不是法律层面蹭热度，就是各种维权维权，法律途径存在效率边界，打败商人的前提是商人思维。他拿了商标权的目的不是逼迫你改名，而是逼迫你拿钱。我们不妨设想一下，如果你真的改了名，结果会如何？我所能预知到的就是商标变得毫无价值，因为这个商标就是完全依附在你个人身上的，这不是Dell也不是爱马仕。为何会倒卖多次？也是因为如此，打的就是时间差，最后的维权结果他们是利润上的100%赢家，这就是他们肆无忌惮的原因。所以尽量压低他们的期待利益，多条腿走路，表明

投币达到上限了？ ['pos']
拿起法律的武器（锤子）锤死他们 ['neg']
如果改名字的话，可以再原来真名后面加jhq ['pos']
我现在才知道敬汉卿是真名。 ['pos']
加油！！！！ ['pos']
加油，顺便别qinghanjing也被抢注了 ['pos']
加油_x1f4aa__x1f4aa__x1f4aa_ ['pos']
加油(ง •̀_•́)ง这几天辛苦了 ['pos']
所有人，团结起来！让一切黑恶势力陷入人民战争的汪洋大海中吧！ ['pos']
牛批 ['pos']
好的 ['pos']
忘了评论了，敬汉卿加油嗷 ['pos']
加油ヽ(`Д´)ﾉ ['pos']
周树人：他们玩玩没有想到，我的真名就叫鲁迅 ['neg']
三连！加油！！打气！！！[点赞][点赞] ['pos']
继续评论 ['pos']
支持维权[点赞] ['pos']
支持 ['pos']
真是应了一句话，人怕出名猪怕壮，希望你能度过难关 ['pos']
加油 ['pos']
死神之子加油 ['pos']
450w粉丝了，等打赢官司，你知道怎么做了吧 ['pos']
加油 ['pos']
打气 ['pos']
敬汉卿加油！！！ ['pos']
加油 ['pos']
这篇推文刚好在写，团中央的，up主可以看看，我刚刚看到了https://mp.weixin.qq.com/s/qZ2AuyCB3hYClerGs7ujJg ['pos']
11111111 ['pos']
三连 ['pos']
敬汉卿你好，我也经常看你的节目，我们也是做商标这块的。刚才查询了一下，目前这个41还不是最要命的，你的35类也被抢注了，现在正在公示，如果35类被抢注，你就不能做广告，彻底凉凉了，希望你能引起共重视，现在赶紧去。35类的商标进行复议 复制自热心网友35类已经在2019.7.13号开始初审公示了，如果在三个月之内，也就是10月13号之前没有人复议的话就通过了，你35类商标如果丢了的话，你彻底就告别这个名字了（复制粘贴评论区） ['neg']
你很出名！ ['pos']
维权，维权！ ['neg']
商……标 ['pos']
111111111 ['pos']
加油鸭 ['pos']
背后即是国家不用怕 ['pos']
加油！！ ['pos']
加油，会成功驳回的 ['pos']
五天

up加油！！！ ['pos']
加油 ['pos']
[黑洞]尖叫 ['pos']
6666666顶顶顶 ['pos']
up主加油[点赞] ['pos']
加油呢 ['pos']
你很出名 ['pos']
加油敬汉卿 ['pos']
支持敬汉卿[粽子][粽子] ['pos']
加油加油支持你 ['pos']
打气打气，不让这些恰烂钱的畜牲有机可乘 ['neg']
厉害了 ['neg']
支持支持。加油 ['pos']
支持 ['pos']
这家公司好像是空壳公司，专门来坑你的 ['pos']
这个事情已经闹大了坐等央视见[doge] ['neg']
这是我从别人那里复制过来。可能敬汉卿没看到所以我复制了过来 ['neg']
1千万我擦 ['pos']
支持维权！！！ ['pos']
加油！敬汉卿，你一定会赢的！ ['pos']
衣服好好看 ['pos']
牛逼嗷 ['pos']
Ddddddd ['pos']
死神之子也是你能讹的？？？ ['pos']
支持敬汉卿合法维权！！[点赞]你很出名！！ ['pos']
这就是个碰瓷套路，靠名人名字，然后遇到个傻子老板，有本事你注册商标叫习*平，然后去国*院 ['neg']
在共青团中央的公众号看见了这件事。 ['pos']
1329w合影 ['pos']
加油加油 ['pos']
给我上去你有知名度_x1f604_ ['pos']
加油 ['pos']
超级喜欢你，很久以前就喜欢了 ['pos']
我们这有个方大同胡辣汤 这几年那个歌手火了然后起诉这个胡辣汤名字侵权  但是方大同胡辣汤好几年了 那个歌手火的时候胡辣汤已经有好久了 那个歌手维权行为是对的 但是我感觉还是有点不太舒服 ['pos']
真辛苦 ['pos']
